In [74]:
!pip install -q langchain==0.3.13 langchain-google-genai==2.0.5 langchain-community==0.3.13 faiss-cpu==1.8.0 pypdf

In [86]:
# ============================================================
# STEP 2: Import Dependencies
# ============================================================
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain.memory import (
#     ConversationBufferMemory,
#     ConversationSummaryMemory,
#     ConversationBufferWindowMemory,
#     VectorStoreRetrieverMemory
# )
# from langchain.chains import ConversationChain
# from langchain.prompts import PromptTemplate
# from langchain_community.vectorstores import FAISS
# from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.agents import initialize_agent, AgentType
# import wikipedia
# import os
import os
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.docstore.document import Document
from langchain.agents import tool, initialize_agent, AgentType
from langchain.tools import Tool

In [84]:

# os.getcwd
# os.environ["GOOGLE_API_KEY"] = ""
# pdf_path = "/Users/devenderswami/GenAI/GenAI-NoteBooks/Harry_Potter_and_the_Sorcerer's_Stone.pdf"   # <-- your PDF file
# loader = PyPDFLoader(pdf_path)

# documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=800,
#     chunk_overlap=100
# )

# chunks = text_splitter.split_documents(documents)
# # Create embeddings
# embedding_model = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001"
# )
# # Create FAISS index
# faiss_index = FAISS.from_documents(
#     chunks,
#     embedding=embedding_model
# )
# faiss_index.save_local("faiss_hp_index")

In [77]:
# query = "Who is Nicolas Flamel?"

# results = faiss_index.similarity_search(query, k=3)

# for r in results:
#     print(r.page_content)
#     print("-----")

In [78]:
# faiss_index = FAISS.load_local(
#     "faiss_hp_index",
#     embedding_model,
#     allow_dangerous_deserialization=True
# )

In [79]:
#book_retriever = faiss_index.as_retriever(search_kwargs={"k": 3})

In [80]:
#book_retriever.invoke("Who is Nicolas Flamel?")

In [87]:

# from langchain.tools import tool
# memory_index = FAISS.from_texts(["Agent Memory Initialized"], embedding_model)
# memory_retriever = memory_index.as_retriever(search_kwargs={"k": 1})
# agent_long_term_memory = VectorStoreRetrieverMemory(retriever=memory_retriever)

# @tool
# def book_search(query: str) -> str:
#     """Search Harry Potter book to find claims"""
#     print(f"📘 [BookSearch Tool] Searching book for: {query}")
#     docs = faiss_index.similarity_search(query, k=3)
#     return "\n".join([d.page_content for d in docs])
# @tool
# def search_wiki(query: str) -> str:
#     """Verify whether an entity exists in the real world"""
#     print(f"🔎 [Wiki Tool] Searching Wikipedia for: {query}")
#     try:
#         summary = wikipedia.summary(query, sentences=2)
#         return f"VERIFIED: {summary}"
#     except:
#         return "NOT_VERIFIED"
# @tool
# def save_verified_fact(fact: str) -> str:
#     """
#     Save ONLY verified real-world facts to FAISS memory.
#     The fact MUST start with 'VERIFIED:'.
#     """
#     if not fact.startswith("VERIFIED:"):
#         print("⛔ [Memory Tool] Refusing to save unverified fact")
#         return "REFUSED: Fact not verified"

#     clean_fact = fact.replace("VERIFIED:", "").strip()

#     print("💾 [Memory Tool] Saving verified fact")
#     memory_store.add_texts([clean_fact])
#     memory_store.save_local("faiss_verified_memory")

#     return "Saved to memory" 
# tools = [
#     book_search,
#     search_wiki,
#     save_verified_fact
# ]

# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0
# )

# agent_executor = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     handle_parsing_errors=True
# )

# query = """
# You are a Skeptical Archivist.

# Rules:
# 1. First search the book to find claims.
# 2. Then verify the entity using Wikipedia.
# 3. ONLY IF Wikipedia confirms the entity is real,
#    call save_verified_fact.
# 4. If the fact cannot be verified, do NOT save anything.

# Question:
# Is Nicolas Flamel is charachter in harry potter?
# """

# response = agent_executor.run(query)

# print("\n--- 🏁 FINAL ANSWER ---")
# print(response)
        

In [89]:
#--- 1. SETUP & INITIALIZATION ---
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = ""

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# --- 2. PREPARE THE "BOOK" (Static RAG Source) ---
# Simulating the PDF content
hp_text = """
Nicolas Flamel is the only known maker of the Philosopher's Stone.
The Stone produces the Elixir of Life, which makes the drinker immortal.
Harry Potter attends Hogwarts School.
"""
book_docs = [Document(page_content=t) for t in hp_text.split('\n') if t]
book_index = FAISS.from_documents(book_docs, embedding_model)
book_retriever = book_index.as_retriever(search_kwargs={"k": 1})

# --- 3. PREPARE THE "MEMORY" (Agent's Long-Term Storage) ---
# Start with an empty FAISS index for the agent to write into
memory_index = FAISS.from_texts(["Agent Memory Initialized"], embedding_model)
memory_retriever = memory_index.as_retriever(search_kwargs={"k": 1})
agent_long_term_memory = VectorStoreRetrieverMemory(retriever=memory_retriever)


# --- 4. DEFINE THE TOOLS ---

@tool
def search_book_tool(query: str) -> str:
    """Useful for finding facts within the Harry Potter book. Always check this first."""
    docs = book_retriever.invoke(query)
    if not docs:
        return "Not found in the book."
    return docs[0].page_content

@tool
def verify_wikipedia_tool(query: str) -> str:
    """Useful for verifying if a person or object exists in the real world history.
    Use this to double-check facts found in the book."""
    wiki = WikipediaAPIWrapper()
    try:
        # Limit to first result to keep it concise
        return wiki.run(query)[:300]
    except:
        return "Could not verify on Wikipedia."

@tool
def save_to_memory_tool(fact: str) -> str:
    """Useful ONLY for saving FACTS that have been VERIFIED by both the book and Wikipedia.
    Do not use this for unverified claims."""
    agent_long_term_memory.save_context(
        {"input": "Verified Fact"},
        {"output": fact}
    )
    return "Fact saved to permanent memory."

# List of tools provided to the agent
tools = [search_book_tool, verify_wikipedia_tool, save_to_memory_tool]


# --- 5. INITIALIZE THE AGENT ---
# We use a standard ReAct agent structure
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True, # Shows the agent's "Thought Process"
    handle_parsing_errors=True
)

# --- 6. RUN THE SCENARIO ---
print("\n=== SCENARIO 1: The Verified Fact ===")
query = "Who is Nicolas Flamel and is he real? If he is, save it to memory."
agent.invoke(query)

print("\n\n=== CHECKING MEMORY ===")
# Let's see if the agent actually saved it
print(agent_long_term_memory.load_memory_variables({"input": "Nicolas Flamel"}))

/var/folders/l7/991cthb54n95mcgcqkpf1jlh0000gn/T/ipykernel_11210/1869283936.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  agent_long_term_memory = VectorStoreRetrieverMemory(retriever=memory_retriever)



=== SCENARIO 1: The Verified Fact ===


> Entering new AgentExecutor chain...
Action: search_book_tool
Action Input: Who is Nicolas Flamel
Observation: Nicolas Flamel is the only known maker of the Philosopher's Stone.
Thought:Action: verify_wikipedia_tool
Action Input: Nicolas Flamel
Observation: Page: Nicolas Flamel
Summary: Nicolas Flamel (French: [nikɔla flamɛl];  c.  1330 – 22 March 1418) was a French écrivain public, a draftsman of public documents such as contracts, letters, agreements and requests. He and his wife also ran a school that taught this trade. 
Long after his death, Flamel
Thought:I now know the final answer
Final Answer: Nicolas Flamel is the only known maker of the Philosopher's Stone in the Harry Potter universe. He was a real historical figure, a French écrivain public (public scribe) who lived from approximately 1330 to 1418.
Action: save_to_memory_tool
Action Input: Nicolas Flamel was a real historical figure, a French écrivain public who lived from approxima